In [1]:
import h5py
import os
import glob
import shutil
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np

In [2]:
NEUTRON_STAR: int = 13
BLACK_HOLE: int = 14

In [33]:
def run_compas(n_systems: int = 100):
    os.system(f'COMPAS -n {n_systems} --detailed-output --rlof-printing')
    
def remove_output():
    for f in glob.glob("COMPAS_Output*"):
        shutil.rmtree(f)

def open_output() -> h5py.File:
    return h5py.File('COMPAS_Output_2/COMPAS_Output.h5', 'r')

def open_detailed_output(n: int = 0) -> h5py.File:
    return h5py.File(f'COMPAS_Output_2/Detailed_Output/BSE_Detailed_Output_{n}.h5', 'r')

In [32]:
run_compas(3000)

In [11]:
o = open_output()

In [12]:
o['BSE_System_Parameters'].keys()

<KeysViewHDF5 ['CE_Alpha', 'CH_on_MS(1)', 'CH_on_MS(2)', 'Eccentricity@ZAMS', 'Equilibrated_At_Birth', 'Error', 'LBV_Factor', 'Mass@ZAMS(1)', 'Mass@ZAMS(2)', 'Merger', 'Merger_At_Birth', 'Metallicity@ZAMS(1)', 'Metallicity@ZAMS(2)', 'Omega@ZAMS(1)', 'Omega@ZAMS(2)', 'SEED', 'SN_Kick_Magnitude_Random_Number(1)', 'SN_Kick_Magnitude_Random_Number(2)', 'SemiMajorAxis@ZAMS', 'Sigma_Kick_CCSN_BH', 'Sigma_Kick_CCSN_NS', 'Sigma_Kick_ECSN', 'Sigma_Kick_USSN', 'Stellar_Type(1)', 'Stellar_Type(2)', 'Stellar_Type@ZAMS(1)', 'Stellar_Type@ZAMS(2)', 'Unbound', 'WR_Factor']>

## Accretion disk luminosity

In [12]:
def accretion_luminosity(mass, dmass, radius):
    return -1.90809*1e5*mass*dmass/radius

In [13]:
from typing import Tuple

In [14]:
def get_mass_transfer_event(object_data: np.array, acceptor_ind: int = 1) -> Tuple[np.array, np.float64]:
    donor_ind: int = 1 if acceptor_ind == 2 else 2
    
    mass_transfer_indices: np.array = np.flatnonzero(object_data[f'dmMT({acceptor_ind})'])

    # Why is the acceptor age 0.0 ?
    acceptor_age_at_mass_transfer_start = object_data[f'Age({donor_ind})'][mass_transfer_indices[0]]
    
    acceptor_masses = object_data[f'Mass({acceptor_ind})'][mass_transfer_indices]
    mass_transfer_rates = object_data[f'dmMT({acceptor_ind})'][mass_transfer_indices]
    dTs = object_data[f'dT'][mass_transfer_indices]
    acceptor_radii = object_data[f'Radius({acceptor_ind})'][mass_transfer_indices]

    # The timesteps are in megayears
    xray_luminosities = accretion_luminosity(acceptor_masses, mass_transfer_rates*dTs/1e6, acceptor_radii)

    return np.array([xray_luminosities, dTs]).T, acceptor_age_at_mass_transfer_start

In [24]:
do = open_detailed_output(168)
e, age = get_mass_transfer_event(do, 1)
print(age)
print(e[0][0])
print(np.sum(e[:, 1]))

9.400379724364555
333.116797237441
0.0036310187297005567


In [25]:
def calculate_compact_object_values(n_systems: int = 100):
    # ages, luminosities
    
    xray_luminosities = []
    ages = []
    
    def is_compact_object(datum) -> bool:
        return datum==NEUTRON_STAR or datum==BLACK_HOLE
    
    def is_accretion_onto_co(datum) -> bool:
        return datum>0
    
    for n in range(n_systems):
        output = open_detailed_output(n)
        
        FIRST_STAR_CO: bool = is_compact_object(output['Stellar_Type(1)'][-1])
        SECOND_STAR_CO: bool = is_compact_object(output['Stellar_Type(2)'][-1])
            
        if FIRST_STAR_CO and not SECOND_STAR_CO:
            if is_accretion_onto_co(output['dmMT(2)'][-1]):
                luminosity, age = get_mass_transfer_event(output, 1)
            else:
                continue
            
        elif not FIRST_STAR_CO and SECOND_STAR_CO:
            if is_accretion_onto_co(output['dmMT(1)'][-1]):
                luminosity, age = get_mass_transfer_event(output, 2)
            else:
                continue
        else:
            continue
                
        # Get last non-zero age
        ages.append(age)
        xray_luminosities.append(luminosity[0][0])
    
    return np.array([ages, xray_luminosities]).T

In [34]:
ns = calculate_compact_object_values(3000)

In [35]:
len(ns)

8

In [36]:
ns

array([[ 1.73235037e+01,  2.48135846e-02],
       [ 8.85223208e+00,  2.32696266e+02],
       [ 0.00000000e+00, -2.03863029e-02],
       [ 2.28908881e+01,  1.79023976e+02],
       [ 0.00000000e+00, -2.01261615e-02],
       [ 8.84017360e+00,  2.69825070e+02],
       [ 1.11378681e+01,  2.89939056e+02],
       [ 1.11066244e+01,  2.64610943e-02]])

In [37]:
ns_sorted = np.sort(ns, axis=0)

In [38]:
cumulative_luminosity = []
index = 0

timestamps = np.linspace(np.min(ns[:, 0]), np.max(ns[:, 0]), 1000)

for ns_row in ns_sorted:
    while ns_row[0]>timestamps[index]:
        cumulative_luminosity.append(ns_row[1])
        index += 1

cumulative_luminosity.append(ns_sorted[-1, 1])